<a href="https://colab.research.google.com/github/samridhi-narwade/Xenone_Prototype/blob/main/Xenone_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Xenone — Team Knowledge Hub
### AMD Slingshot Hackathon | Team Xenone | Leader: Samridhi Narwade

> **"Preserving 5 years of club knowledge in 5 clicks"**

This notebook implements the **complete Xenone system** with all three core features:
1. 📌 **Passive Knowledge Capture** — Simulate Slack/Discord emoji-based capture
2. 🤖 **AI-Powered Q&A with Confidence Scoring** — RAG using ChromaDB + Ollama/LLM
3. 📄 **One-Click Exit Brief Generator** — Auto-generate structured PDF handoff documents

---
### 🗺️ Notebook Structure
| Step | Section | What It Does |
|------|---------|----------|
| 1 | Install Dependencies | Install all required libraries |
| 2 | Setup & Configuration | Configure API keys, paths |
| 3 | Knowledge Base (ChromaDB) | Vector DB for storing team knowledge |
| 4 | Passive Capture Module | Simulate Slack/Discord capture with emoji |
| 5 | AI Q&A Module | RAG pipeline with confidence scoring |
| 6 | Exit Brief Generator | PDF generation from captured knowledge |
| 7 | Streamlit Dashboard | (Optional) Full UI demo |
| 8 | Full Demo | End-to-end walkthrough |

---
⚠️ **Runtime Required:** Go to `Runtime > Change runtime type > GPU (T4)` for best performance.

## 📦 STEP 1: Install All Dependencies

In [1]:
# ============================================================
# STEP 1: Install all required packages
# Runtime: ~2-3 minutes
# ============================================================
print("Installing dependencies... Please wait (~2-3 mins)")

!pip install -q chromadb sentence-transformers
!pip install -q transformers torch accelerate
!pip install -q fpdf2 reportlab
!pip install -q streamlit pyngrok
!pip install -q langchain langchain-community
!pip install -q slack-bolt discord.py
!pip install -q gradio  # For interactive UI inside Colab

print("\n✅ All dependencies installed successfully!")

Installing dependencies... Please wait (~2-3 mins)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
E

## ⚙️ STEP 2: Configuration & Setup

In [2]:
# ============================================================
# STEP 2A: Imports and Global Configuration
# ============================================================
import os
import json
import datetime
import uuid
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field, asdict
from pathlib import Path

import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# ── Global Config ──────────────────────────────────────────
CONFIG = {
    "team_name": "Xenone Demo Team",          # Change to your club/team name
    "db_path": "/content/xenone_db",          # ChromaDB storage path
    "collection_name": "team_knowledge",       # ChromaDB collection name
    "embedding_model": "all-MiniLM-L6-v2",    # Fast, accurate embedding model
    "llm_model": "google/flan-t5-base",        # LLM for answer generation (free, no API key)
    "confidence_high_threshold": 0.75,         # Score >= 0.75 → High confidence
    "confidence_med_threshold": 0.50,          # Score >= 0.50 → Medium confidence
    "top_k_results": 3,                        # Number of sources to retrieve
    "capture_emoji": "📌",                     # Emoji that triggers capture
}

os.makedirs(CONFIG["db_path"], exist_ok=True)
os.makedirs("/content/xenone_exports", exist_ok=True)

print(f"✅ Config loaded for team: {CONFIG['team_name']}")
print(f"📁 Database path: {CONFIG['db_path']}")
print(f"🏷️  Capture emoji: {CONFIG['capture_emoji']}")

✅ Config loaded for team: Xenone Demo Team
📁 Database path: /content/xenone_db
🏷️  Capture emoji: 📌


In [3]:
# ============================================================
# STEP 2B: Data Models
# These represent the core data structures in Xenone
# ============================================================

@dataclass
class CapturedMessage:
    """Represents a captured knowledge item from Slack/Discord"""
    id: str = field(default_factory=lambda: str(uuid.uuid4())[:8])
    content: str = ""
    author: str = ""
    channel: str = ""
    platform: str = "slack"          # slack | discord
    timestamp: str = field(default_factory=lambda: datetime.datetime.now().isoformat())
    tags: List[str] = field(default_factory=list)
    category: str = "general"        # decision | lesson | warning | context

@dataclass
class QueryResult:
    """Represents an AI Q&A response with confidence scoring"""
    question: str = ""
    answer: str = ""
    confidence_score: float = 0.0
    confidence_label: str = ""       # High | Medium | Low
    source_count: int = 0
    sources: List[Dict] = field(default_factory=list)
    timestamp: str = field(default_factory=lambda: datetime.datetime.now().isoformat())

print("✅ Data models defined: CapturedMessage, QueryResult")
print("   - CapturedMessage: stores team knowledge items")
print("   - QueryResult: stores Q&A responses with confidence")

✅ Data models defined: CapturedMessage, QueryResult
   - CapturedMessage: stores team knowledge items
   - QueryResult: stores Q&A responses with confidence


## 🗄️ STEP 3: Knowledge Base (ChromaDB Vector Database)

In [4]:
# ============================================================
# STEP 3: Initialize ChromaDB Vector Knowledge Base
# ChromaDB stores messages as vector embeddings for
# semantic (meaning-based) search
# ============================================================

class XenoneKnowledgeBase:
    """
    Core knowledge base for Xenone.
    Uses ChromaDB for vector storage and sentence-transformers for embeddings.
    """

    def __init__(self, config: dict):
        self.config = config
        print("🔄 Loading embedding model (first time takes ~30s)...")

        # Load embedding model — converts text to vectors
        self.embedding_model = SentenceTransformer(config["embedding_model"])

        # Initialize ChromaDB client (persistent storage)
        self.client = chromadb.PersistentClient(path=config["db_path"])

        # Get or create the knowledge collection
        self.collection = self.client.get_or_create_collection(
            name=config["collection_name"],
            metadata={"hnsw:space": "cosine"}  # Cosine similarity for better semantic search
        )

        print(f"✅ ChromaDB initialized: '{config['collection_name']}'")
        print(f"📊 Current knowledge items: {self.collection.count()}")

    def add_message(self, msg: CapturedMessage) -> bool:
        """Add a captured message to the vector knowledge base"""
        try:
            # Generate embedding vector from message content
            embedding = self.embedding_model.encode(msg.content).tolist()

            # Store in ChromaDB with full metadata
            self.collection.add(
                ids=[msg.id],
                embeddings=[embedding],
                documents=[msg.content],
                metadatas=[{
                    "author": msg.author,
                    "channel": msg.channel,
                    "platform": msg.platform,
                    "timestamp": msg.timestamp,
                    "category": msg.category,
                    "tags": ",".join(msg.tags)
                }]
            )
            return True
        except Exception as e:
            print(f"❌ Error adding message: {e}")
            return False

    def search(self, query: str, n_results: int = 3) -> List[Dict]:
        """Semantic search — find relevant messages by meaning, not just keywords"""
        if self.collection.count() == 0:
            return []

        # Encode the query into a vector
        query_embedding = self.embedding_model.encode(query).tolist()

        # Search ChromaDB for similar vectors
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=min(n_results, self.collection.count()),
            include=["documents", "metadatas", "distances"]
        )

        # Format and return results
        formatted = []
        for i in range(len(results["ids"][0])):
            similarity_score = 1 - results["distances"][0][i]  # Convert distance to similarity
            formatted.append({
                "id": results["ids"][0][i],
                "content": results["documents"][0][i],
                "metadata": results["metadatas"][0][i],
                "similarity": round(similarity_score, 3)
            })
        return formatted

    def get_all(self, category: str = None) -> List[Dict]:
        """Retrieve all knowledge items (optionally filtered by category)"""
        if self.collection.count() == 0:
            return []
        results = self.collection.get(include=["documents", "metadatas"])
        items = []
        for i in range(len(results["ids"])):
            item = {
                "id": results["ids"][i],
                "content": results["documents"][i],
                "metadata": results["metadatas"][i]
            }
            if category is None or results["metadatas"][i].get("category") == category:
                items.append(item)
        return items

    def stats(self) -> Dict:
        """Return statistics about the knowledge base"""
        all_items = self.get_all()
        categories = {}
        for item in all_items:
            cat = item["metadata"].get("category", "general")
            categories[cat] = categories.get(cat, 0) + 1
        return {
            "total": len(all_items),
            "by_category": categories
        }


# Initialize the knowledge base
kb = XenoneKnowledgeBase(CONFIG)
print("\n🎉 Knowledge Base ready!")

🔄 Loading embedding model (first time takes ~30s)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ ChromaDB initialized: 'team_knowledge'
📊 Current knowledge items: 0

🎉 Knowledge Base ready!


## 📌 STEP 4: Passive Capture Module
Simulates how the Slack/Discord bot captures messages when a user reacts with 📌

In [5]:
# ============================================================
# STEP 4A: Passive Capture Engine
# In production: triggered by Slack Events API / Discord.py
# Here: simulated with the same logic
# ============================================================

class XenoneCapture:
    """
    Handles passive knowledge capture.
    In production: this runs inside the Slack/Discord bot event handler.
    In Colab: call capture_message() directly to simulate the bot.
    """

    def __init__(self, knowledge_base: XenoneKnowledgeBase):
        self.kb = knowledge_base
        self.category_keywords = {
            "decision": ["decided", "chose", "selected", "agreed", "going with", "will use", "picked"],
            "lesson":   ["learned", "lesson", "note", "remember", "tip", "turns out", "realize", "discovered"],
            "warning":  ["warning", "caution", "avoid", "don't", "never", "failed", "bug", "issue", "problem", "mistake"],
            "context":  ["because", "reason", "background", "history", "context", "why", "since"]
        }

    def _auto_categorize(self, text: str) -> str:
        """Automatically categorize a message based on keyword signals"""
        text_lower = text.lower()
        scores = {cat: 0 for cat in self.category_keywords}
        for cat, keywords in self.category_keywords.items():
            for kw in keywords:
                if kw in text_lower:
                    scores[cat] += 1
        best = max(scores, key=scores.get)
        return best if scores[best] > 0 else "general"

    def capture_message(
        self,
        content: str,
        author: str,
        channel: str = "#general",
        platform: str = "slack",
        tags: List[str] = None,
        category: str = None  # If None, auto-detect
    ) -> CapturedMessage:
        """
        Main capture function — called when user reacts with 📌 emoji.

        In production Slack bot:
          @app.event("reaction_added")
          def handle_pin(event, say):
              if event['reaction'] == 'pushpin':
                  capture_message(event['item']['text'], ...)
        """
        msg = CapturedMessage(
            content=content,
            author=author,
            channel=channel,
            platform=platform,
            tags=tags or [],
            category=category or self._auto_categorize(content)
        )

        success = self.kb.add_message(msg)

        if success:
            # In production: bot sends this as a reply in Slack/Discord
            print(f"✅ Saved to knowledge base")
            print(f"   📝 Content  : {content[:80]}{'...' if len(content) > 80 else ''}")
            print(f"   👤 Author   : {author}")
            print(f"   📍 Channel  : {channel} ({platform})")
            print(f"   🏷️  Category : {msg.category}")
            print(f"   🆔 ID       : {msg.id}")
            print(f"   📊 Total KB : {self.kb.collection.count()} items")

        return msg


# Initialize capture module
capturer = XenoneCapture(kb)
print("✅ Passive Capture module ready!")
print("   Usage: capturer.capture_message(content, author, channel)")

✅ Passive Capture module ready!
   Usage: capturer.capture_message(content, author, channel)


In [6]:
# ============================================================
# STEP 4B: Seed Demo Data
# Simulates an entrepreneurship club capturing messages
# over a semester — covers decisions, lessons, warnings
# ============================================================

DEMO_MESSAGES = [
    # DECISIONS
    {
        "content": "We decided to go with PrintMaster as our printing vendor. They offer 3-day turnaround vs 7-day from CompuPrint, and the per-unit cost is 12% lower for orders above 500 units. Email confirmation received from Rohan at PrintMaster.",
        "author": "Samridhi", "channel": "#vendor-decisions",
        "category": "decision", "tags": ["vendor", "printing", "budget"]
    },
    {
        "content": "Q3 marketing budget decided: Total ₹45,000. Allocation — LinkedIn ads: ₹18,000, YouTube: ₹15,000, College fest banners: ₹12,000. Decision was made after comparing last year's ROI where LinkedIn gave 3x more leads than Instagram.",
        "author": "Aryan", "channel": "#budget-planning",
        "category": "decision", "tags": ["budget", "marketing", "Q3"]
    },
    {
        "content": "Selected Notion for project documentation over Confluence. Reasons: free tier is sufficient, all members already have accounts, and Confluence requires IT admin access we don't have.",
        "author": "Priya", "channel": "#tools",
        "category": "decision", "tags": ["tools", "documentation"]
    },
    {
        "content": "Tech stack chosen for the competition portal: React frontend, FastAPI backend, PostgreSQL database hosted on Supabase. Chose Supabase over Firebase because it's open-source and we have more SQL experience.",
        "author": "Dev Team", "channel": "#tech-decisions",
        "category": "decision", "tags": ["tech-stack", "portal", "backend"]
    },
    # LESSONS LEARNED
    {
        "content": "Lesson learned: Always book the venue at least 6 weeks in advance. This year we tried 3 weeks out and our first 2 choices were unavailable. The last-minute scramble caused a 2-week delay in event promotion.",
        "author": "Riya", "channel": "#event-planning",
        "category": "lesson", "tags": ["venue", "events", "timeline"]
    },
    {
        "content": "Turns out sending sponsorship emails on Tuesday or Wednesday mornings gets ~40% better response rates compared to Monday or Friday. Discovered this after tracking 60+ outreach emails over 2 months.",
        "author": "Kabir", "channel": "#sponsorships",
        "category": "lesson", "tags": ["sponsorship", "outreach", "email"]
    },
    {
        "content": "We learned that asking judges to fill a paper feedback form at the end of presentations leads to incomplete responses. Switching to a Google Form QR code displayed on screen improved form completion from 40% to 90%.",
        "author": "Meera", "channel": "#competition-management",
        "category": "lesson", "tags": ["judges", "feedback", "competition"]
    },
    # WARNINGS
    {
        "content": "WARNING: Never book the campus auditorium without written confirmation from the Admin Office, even if verbal permission is given. We lost our booking once because verbal approval wasn't recorded in their system.",
        "author": "Samridhi", "channel": "#warnings-pitfalls",
        "category": "warning", "tags": ["venue", "bookings", "admin"]
    },
    {
        "content": "Avoid using free Canva templates for official branding — two other clubs used the same template last year and our brochure looked identical to theirs. Use the club brand kit stored in the Drive shared folder.",
        "author": "Design Team", "channel": "#branding",
        "category": "warning", "tags": ["design", "branding", "canva"]
    },
    {
        "content": "Bug: The registration portal doesn't handle special characters in team names (e.g. '&' or '/'). This caused 12 teams to get registration errors during last year's hackathon. Need to add input sanitization.",
        "author": "Aditya", "channel": "#tech-bugs",
        "category": "warning", "tags": ["bug", "portal", "registration"]
    },
    # CONTEXT
    {
        "content": "Background on why we moved away from Instagram for event promotion: engagement dropped 60% after the algorithm change in Feb 2024. LinkedIn posts about entrepreneurship events get organic reach from alumni networks which drives better registrations.",
        "author": "Social Media Lead", "channel": "#social-media",
        "category": "context", "tags": ["instagram", "linkedin", "social-media"]
    },
    {
        "content": "Context: The annual budget approval from the Student Council requires: 1) itemized budget sheet, 2) previous year expense report, 3) faculty advisor signature. Submit at least 4 weeks before event. Finance team contact: Prof. Sharma (finance@college.edu)",
        "author": "Treasurer", "channel": "#finance",
        "category": "context", "tags": ["budget", "approval", "student-council"]
    }
]

print("📥 Loading demo knowledge into the database...\n")

for i, msg_data in enumerate(DEMO_MESSAGES, 1):
    msg = capturer.capture_message(**msg_data)
    print()

stats = kb.stats()
print("\n" + "="*55)
print(f"📊 KNOWLEDGE BASE SUMMARY")
print(f"   Total items: {stats['total']}")
for cat, count in stats['by_category'].items():
    print(f"   {cat.capitalize():12}: {count} items")
print("="*55)

📥 Loading demo knowledge into the database...

✅ Saved to knowledge base
   📝 Content  : We decided to go with PrintMaster as our printing vendor. They offer 3-day turna...
   👤 Author   : Samridhi
   📍 Channel  : #vendor-decisions (slack)
   🏷️  Category : decision
   🆔 ID       : 69384920
   📊 Total KB : 1 items

✅ Saved to knowledge base
   📝 Content  : Q3 marketing budget decided: Total ₹45,000. Allocation — LinkedIn ads: ₹18,000, ...
   👤 Author   : Aryan
   📍 Channel  : #budget-planning (slack)
   🏷️  Category : decision
   🆔 ID       : aab74eb6
   📊 Total KB : 2 items

✅ Saved to knowledge base
   📝 Content  : Selected Notion for project documentation over Confluence. Reasons: free tier is...
   👤 Author   : Priya
   📍 Channel  : #tools (slack)
   🏷️  Category : decision
   🆔 ID       : d2fabbbf
   📊 Total KB : 3 items

✅ Saved to knowledge base
   📝 Content  : Tech stack chosen for the competition portal: React frontend, FastAPI backend, P...
   👤 Author   : Dev Team
   📍 Chann

In [7]:
# ============================================================
# STEP 4C: Interactive Capture — Add YOUR OWN messages!
# ============================================================

print("📌 Add your own knowledge item:")
print("-" * 40)

# ── EDIT THESE VALUES ──────────────────────────────────────
MY_MESSAGE = "We decided to use Google Meet instead of Zoom for all virtual meetings because Zoom has a 40-minute limit on free accounts and Meet integrates directly with our Google Workspace."
MY_AUTHOR  = "Your Name"
MY_CHANNEL = "#tools"
MY_TAGS    = ["meetings", "tools", "decision"]
# ───────────────────────────────────────────────────────────

capturer.capture_message(
    content=MY_MESSAGE,
    author=MY_AUTHOR,
    channel=MY_CHANNEL,
    tags=MY_TAGS
)

print(f"\n✅ Total knowledge items: {kb.collection.count()}")

📌 Add your own knowledge item:
----------------------------------------
✅ Saved to knowledge base
   📝 Content  : We decided to use Google Meet instead of Zoom for all virtual meetings because Z...
   👤 Author   : Your Name
   📍 Channel  : #tools (slack)
   🏷️  Category : decision
   🆔 ID       : 2c287ee4
   📊 Total KB : 13 items

✅ Total knowledge items: 13


## 🤖 STEP 5: AI-Powered Q&A with Confidence Scoring
RAG (Retrieval-Augmented Generation) pipeline that answers from YOUR team's knowledge

In [8]:
# ============================================================
# STEP 5A: Load Language Model
# Using google/flan-t5-base — free, no API key needed,
# runs on CPU/GPU in Colab
# In production: replace with Ollama (Llama 3) on AMD MI300X
# ============================================================

from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

print("🔄 Loading language model (first time: ~60 seconds)...")
print("   Model: google/flan-t5-base (free, no API key needed)")
print("   In production: Ollama + Llama 3 on AMD MI300X\n")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"   Running on: {DEVICE.upper()}")

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base").to(DEVICE)

print("\n✅ Language model loaded!")

🔄 Loading language model (first time: ~60 seconds)...
   Model: google/flan-t5-base (free, no API key needed)
   In production: Ollama + Llama 3 on AMD MI300X

   Running on: CPU


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


✅ Language model loaded!


In [9]:
# ============================================================
# STEP 5B: RAG Q&A Engine with Confidence Scoring
# This is Xenone's core intelligence layer
# ============================================================

class XenoneQA:
    """
    RAG-based Q&A engine.
    1. Retrieve → ChromaDB finds relevant messages semantically
    2. Augment  → Build a context prompt from retrieved messages
    3. Generate → LLM answers ONLY from the retrieved context
    4. Score    → Confidence score based on source quality
    """

    def __init__(self, knowledge_base: XenoneKnowledgeBase, model, tokenizer, config: dict):
        self.kb = knowledge_base
        self.model = model
        self.tokenizer = tokenizer
        self.config = config

    def _calculate_confidence(self, sources: List[Dict]) -> Tuple[float, str]:
        """
        Confidence scoring logic:
        - Based on semantic similarity of retrieved sources to the query
        - More high-quality sources → higher confidence
        - Explicitly flags weak answers (Xenone differentiator)
        """
        if not sources:
            return 0.0, "No Sources"

        # Weighted average — top result counts more
        weights = [0.5, 0.3, 0.2][:len(sources)]
        score = sum(s["similarity"] * w for s, w in zip(sources, weights))
        score = min(score * 1.2, 1.0)  # Slight boost, capped at 1.0

        if score >= self.config["confidence_high_threshold"]:
            label = "High"
        elif score >= self.config["confidence_med_threshold"]:
            label = "Medium"
        else:
            label = "Low — consider documenting this topic!"

        return round(score, 3), label

    def _build_prompt(self, question: str, sources: List[Dict]) -> str:
        """Build a RAG prompt — LLM only sees the retrieved context, not general knowledge"""
        context_parts = []
        for i, src in enumerate(sources, 1):
            meta = src["metadata"]
            context_parts.append(
                f"[Source {i} | {meta.get('author','?')} | {meta.get('channel','?')} | {meta.get('timestamp','?')[:10]}]\n"
                f"{src['content']}"
            )
        context = "\n\n".join(context_parts)

        return (
            f"You are a helpful assistant for a student club. "
            f"Answer the question using ONLY the provided team knowledge. "
            f"If the context doesn't contain enough information, say so clearly.\n\n"
            f"Team Knowledge:\n{context}\n\n"
            f"Question: {question}\n"
            f"Answer:"
        )

    def _generate_answer(self, prompt: str) -> str:
        """Generate answer using the LLM"""
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            max_length=512,
            truncation=True
        ).to(DEVICE)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=150,
                num_beams=4,
                early_stopping=True,
                no_repeat_ngram_size=2
            )

        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def ask(self, question: str) -> QueryResult:
        """
        Main Q&A method — maps to /ask command in Slack/Discord
        Usage: /ask what was our marketing budget decision?
        """
        print(f"\n🔍 Searching knowledge base for: '{question}'")

        # Step 1: Retrieve relevant sources
        sources = self.kb.search(question, n_results=self.config["top_k_results"])

        if not sources:
            return QueryResult(
                question=question,
                answer="No relevant information found in the team's knowledge base. Consider documenting this topic!",
                confidence_score=0.0,
                confidence_label="No Sources",
                source_count=0,
                sources=[]
            )

        # Step 2: Calculate confidence score
        confidence_score, confidence_label = self._calculate_confidence(sources)

        # Step 3: Build RAG prompt and generate answer
        prompt = self._build_prompt(question, sources)
        answer = self._generate_answer(prompt)

        return QueryResult(
            question=question,
            answer=answer,
            confidence_score=confidence_score,
            confidence_label=confidence_label,
            source_count=len(sources),
            sources=sources
        )

    def display_result(self, result: QueryResult):
        """Pretty-print the Q&A result (simulates bot response in Slack)"""
        # Confidence bar
        bar_filled = int(result.confidence_score * 20)
        bar = "█" * bar_filled + "░" * (20 - bar_filled)

        confidence_emoji = {"High": "🟢", "Medium": "🟡", "Low": "🔴"}.get(
            result.confidence_label.split(" ")[0], "⚪"
        )

        print("\n" + "═"*60)
        print(f"❓ QUESTION: {result.question}")
        print("─"*60)
        print(f"💬 ANSWER:")
        print(f"   {result.answer}")
        print("─"*60)
        print(f"📊 CONFIDENCE: {confidence_emoji} {result.confidence_label}")
        print(f"   [{bar}] {result.confidence_score:.0%}")
        print(f"   Sources: {result.source_count} messages")
        print("─"*60)
        print("📎 SOURCES (click to verify):")
        for i, src in enumerate(result.sources, 1):
            meta = src["metadata"]
            sim_pct = f"{src['similarity']:.0%}"
            print(f"   [{i}] {meta.get('channel','?')} by {meta.get('author','?')} "
                  f"on {meta.get('timestamp','?')[:10]} (relevance: {sim_pct})")
            print(f"       '{src['content'][:80]}...'")
        print("═"*60)


# Initialize QA engine
qa = XenoneQA(kb, model, tokenizer, CONFIG)
print("✅ Q&A Engine ready!")
print("   Usage: result = qa.ask('your question here')")

✅ Q&A Engine ready!
   Usage: result = qa.ask('your question here')


In [10]:
# ============================================================
# STEP 5C: Run Sample Queries
# These simulate /ask commands from Slack/Discord
# ============================================================

sample_questions = [
    "Why did we choose our printing vendor?",
    "What was the marketing budget decision?",
    "What should we avoid when booking venues?",
    "How do I get budget approval from the student council?"
]

print("🚀 Running sample /ask queries...\n")

for q in sample_questions:
    result = qa.ask(q)
    qa.display_result(result)
    print()

🚀 Running sample /ask queries...


🔍 Searching knowledge base for: 'Why did we choose our printing vendor?'

════════════════════════════════════════════════════════════
❓ QUESTION: Why did we choose our printing vendor?
────────────────────────────────────────────────────────────
💬 ANSWER:
   They offer 3-day turnaround vs 7-day from CompuPrint
────────────────────────────────────────────────────────────
📊 CONFIDENCE: 🟡 Medium
   [██████████░░░░░░░░░░] 50%
   Sources: 3 messages
────────────────────────────────────────────────────────────
📎 SOURCES (click to verify):
   [1] #vendor-decisions by Samridhi on 2026-02-28 (relevance: 62%)
       'We decided to go with PrintMaster as our printing vendor. They offer 3-day turna...'
   [2] #branding by Design Team on 2026-02-28 (relevance: 25%)
       'Avoid using free Canva templates for official branding — two other clubs used th...'
   [3] #tech-decisions by Dev Team on 2026-02-28 (relevance: 18%)
       'Tech stack chosen for the competit

In [11]:
# ============================================================
# STEP 5D: Ask YOUR OWN question
# ============================================================

# ── EDIT THIS ──────────────────────────────────────────────
MY_QUESTION = "What tech stack are we using for the portal?"
# ───────────────────────────────────────────────────────────

result = qa.ask(MY_QUESTION)
qa.display_result(result)


🔍 Searching knowledge base for: 'What tech stack are we using for the portal?'

════════════════════════════════════════════════════════════
❓ QUESTION: What tech stack are we using for the portal?
────────────────────────────────────────────────────────────
💬 ANSWER:
   React, FastAPI, PostgreSQL
────────────────────────────────────────────────────────────
📊 CONFIDENCE: 🔴 Low — consider documenting this topic!
   [█████████░░░░░░░░░░░] 46%
   Sources: 3 messages
────────────────────────────────────────────────────────────
📎 SOURCES (click to verify):
   [1] #tech-decisions by Dev Team on 2026-02-28 (relevance: 51%)
       'Tech stack chosen for the competition portal: React frontend, FastAPI backend, P...'
   [2] #tools by Priya on 2026-02-28 (relevance: 31%)
       'Selected Notion for project documentation over Confluence. Reasons: free tier is...'
   [3] #tech-bugs by Aditya on 2026-02-28 (relevance: 18%)
       'Bug: The registration portal doesn't handle special characters in te

## 📄 STEP 6: One-Click Exit Brief Generator
Generates a structured PDF handoff document from all captured team knowledge

In [12]:
# ============================================================
# STEP 6A: Exit Brief Generator
# Produces a structured PDF compiling all captured knowledge
# Triggered by: "Generate Exit Brief" button on dashboard
#               OR /handoff slash command in Slack
# ============================================================

from fpdf import FPDF
import textwrap

class XenoneExitBrief:
    """
    Generates a structured PDF exit brief from the knowledge base.
    Sections: Key Decisions | Lessons Learned | Warnings | Unfinished Threads
    """

    SECTION_CONFIG = {
        "decision": {
            "title": "KEY DECISIONS",
            "emoji": "⚡",
            "color": (30, 90, 180),      # Blue
            "bg": (235, 242, 255)
        },
        "lesson": {
            "title": "LESSONS LEARNED",
            "emoji": "📚",
            "color": (20, 140, 60),       # Green
            "bg": (235, 255, 240)
        },
        "warning": {
            "title": "WARNINGS & PITFALLS",
            "emoji": "⚠️",
            "color": (200, 80, 20),       # Orange
            "bg": (255, 245, 235)
        },
        "general": {
            "title": "CONTEXT & BACKGROUND",
            "emoji": "📎",
            "color": (100, 50, 160),      # Purple
            "bg": (248, 240, 255)
        },
        "context": {
            "title": "CONTEXT & BACKGROUND",
            "emoji": "📎",
            "color": (100, 50, 160),
            "bg": (248, 240, 255)
        }
    }

    def __init__(self, knowledge_base: XenoneKnowledgeBase, team_name: str):
        self.kb = knowledge_base
        self.team_name = team_name

    def _make_pdf(self, all_items: List[Dict], project_name: str) -> FPDF:
        """Build the complete PDF document"""
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()

        # ── Cover / Header ─────────────────────────────────
        pdf.set_fill_color(15, 23, 42)   # Dark navy
        pdf.rect(0, 0, 210, 50, 'F')

        pdf.set_font("Helvetica", "B", 22)
        pdf.set_text_color(255, 255, 255)
        pdf.set_xy(10, 10)
        pdf.cell(0, 10, "PROJECT EXIT BRIEF", ln=True, align="C")

        pdf.set_font("Helvetica", "", 13)
        pdf.set_text_color(200, 220, 255)
        pdf.set_xy(10, 23)
        pdf.cell(0, 8, f"{project_name}  |  {self.team_name}", ln=True, align="C")

        pdf.set_font("Helvetica", "", 10)
        pdf.set_text_color(150, 180, 220)
        pdf.set_xy(10, 34)
        pdf.cell(0, 8, f"Generated by Xenone  |  {datetime.date.today().strftime('%B %d, %Y')}", ln=True, align="C")

        # ── Summary Stats Bar ───────────────────────────────
        pdf.set_xy(0, 55)
        stats = {}
        for item in all_items:
            cat = item["metadata"].get("category", "general")
            stats[cat] = stats.get(cat, 0) + 1

        pdf.set_fill_color(248, 250, 252)
        pdf.rect(10, 58, 190, 25, 'F')

        stat_labels = [
            ("Total Items", len(all_items)),
            ("Decisions", stats.get("decision", 0)),
            ("Lessons", stats.get("lesson", 0)),
            ("Warnings", stats.get("warning", 0)),
        ]

        for i, (label, count) in enumerate(stat_labels):
            x = 15 + i * 47
            pdf.set_font("Helvetica", "B", 18)
            pdf.set_text_color(15, 23, 42)
            pdf.set_xy(x, 61)
            pdf.cell(40, 8, str(count), align="C")
            pdf.set_font("Helvetica", "", 8)
            pdf.set_text_color(100, 120, 140)
            pdf.set_xy(x, 71)
            pdf.cell(40, 5, label.upper(), align="C")

        pdf.set_xy(10, 88)
        pdf.set_font("Helvetica", "I", 9)
        pdf.set_text_color(120, 140, 160)
        pdf.cell(0, 6, "This document was auto-generated from team knowledge captured via Xenone. Verify critical decisions with original sources.", ln=True)
        pdf.ln(4)

        # ── Sections ─────────────────────────────────────────
        section_order = ["decision", "lesson", "warning", "context", "general"]
        seen_sections = set()

        for cat in section_order:
            if cat in seen_sections:
                continue
            # Merge 'context' and 'general'
            if cat == "general":
                items = [i for i in all_items if i["metadata"].get("category") in ("general", "context")]
            else:
                items = [i for i in all_items if i["metadata"].get("category") == cat]

            if not items:
                continue

            seen_sections.add(cat)
            seen_sections.add("context")  # Prevent double context section

            cfg = self.SECTION_CONFIG.get(cat, self.SECTION_CONFIG["general"])
            r, g, b = cfg["color"]
            br, bg, bb = cfg["bg"]

            # Section header
            pdf.set_fill_color(r, g, b)
            pdf.rect(10, pdf.get_y(), 190, 12, 'F')
            pdf.set_font("Helvetica", "B", 13)
            pdf.set_text_color(255, 255, 255)
            pdf.set_x(10)
            pdf.cell(190, 12, f"  {cfg['title']}  ({len(items)} items)", ln=True)
            pdf.ln(3)

            # Items
            for item in items:
                meta = item["metadata"]
                y_before = pdf.get_y()

                # Item background box
                pdf.set_fill_color(br, bg, bb)
                pdf.rect(10, y_before, 190, 30, 'F')  # approximate height

                # Author / Channel / Date line
                pdf.set_font("Helvetica", "B", 8)
                pdf.set_text_color(r, g, b)
                pdf.set_xy(12, y_before + 2)
                date_str = meta.get("timestamp", "")[:10]
                pdf.cell(0, 5,
                    f"{meta.get('author','?')}  |  {meta.get('channel','?')}  |  {date_str}"
                    f"  |  Tags: {meta.get('tags','')}",
                    ln=True
                )

                # Content
                pdf.set_font("Helvetica", "", 10)
                pdf.set_text_color(20, 30, 50)
                pdf.set_x(12)
                wrapped = textwrap.fill(item["content"], width=100)
                for line in wrapped.split("\n"):
                    pdf.cell(0, 5, line, ln=True)

                pdf.ln(4)

            pdf.ln(4)

        # ── Footer ─────────────────────────────────────────
        pdf.set_y(-20)
        pdf.set_font("Helvetica", "I", 9)
        pdf.set_text_color(140, 150, 160)
        pdf.cell(0, 10,
            f"Xenone | AMD Slingshot Hackathon | Generated {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')} | Page {pdf.page_no()}",
            align="C"
        )

        return pdf

    def generate(
        self,
        project_name: str = "Q3 Project Handoff",
        output_path: str = "/content/xenone_exports/exit_brief.pdf"
    ) -> str:
        """Generate the exit brief PDF — one click in the dashboard"""
        print(f"\n📄 Generating Exit Brief...")
        print(f"   Project: {project_name}")

        all_items = self.kb.get_all()
        if not all_items:
            print("❌ No knowledge items found. Capture some messages first!")
            return None

        print(f"   Processing {len(all_items)} knowledge items...")
        pdf = self._make_pdf(all_items, project_name)
        pdf.output(output_path)

        file_size = os.path.getsize(output_path) / 1024
        print(f"\n✅ Exit Brief generated!")
        print(f"   📁 Saved to: {output_path}")
        print(f"   📦 File size: {file_size:.1f} KB")
        print(f"   📊 Items included: {len(all_items)}")
        print(f"\n💡 To download: Files panel (📁) on the left → Right-click → Download")

        return output_path


# Initialize Exit Brief Generator
exit_brief_gen = XenoneExitBrief(kb, CONFIG["team_name"])
print("✅ Exit Brief Generator ready!")
print("   Usage: exit_brief_gen.generate('Project Name')")

✅ Exit Brief Generator ready!
   Usage: exit_brief_gen.generate('Project Name')


In [13]:
# ============================================================
# STEP 6A FIX: Replace the _make_pdf method in XenoneExitBrief
# Changes: adds Unicode font + sanitize helper to handle ₹ and other symbols
# ============================================================

import unicodedata

class XenoneExitBrief:

    SECTION_CONFIG = {
        "decision": {"title": "KEY DECISIONS", "color": (30, 90, 180), "bg": (235, 242, 255)},
        "lesson":   {"title": "LESSONS LEARNED", "color": (20, 140, 60), "bg": (235, 255, 240)},
        "warning":  {"title": "WARNINGS & PITFALLS", "color": (200, 80, 20), "bg": (255, 245, 235)},
        "general":  {"title": "CONTEXT & BACKGROUND", "color": (100, 50, 160), "bg": (248, 240, 255)},
        "context":  {"title": "CONTEXT & BACKGROUND", "color": (100, 50, 160), "bg": (248, 240, 255)},
    }

    def __init__(self, knowledge_base, team_name: str):
        self.kb = knowledge_base
        self.team_name = team_name

    def _sanitize(self, text: str) -> str:
        """Replace problematic Unicode characters with ASCII equivalents"""
        replacements = {
            '₹': 'Rs.',   # Rupee sign
            '€': 'EUR',
            '£': 'GBP',
            '©': '(c)',
            '®': '(R)',
            '™': '(TM)',
            '\u2019': "'",   # Right single quote
            '\u2018': "'",   # Left single quote
            '\u201C': '"',   # Left double quote
            '\u201D': '"',   # Right double quote
            '\u2014': '--',  # Em dash
            '\u2013': '-',   # En dash
            '\u2022': '*',   # Bullet
            '\u2026': '...',  # Ellipsis
        }
        for char, replacement in replacements.items():
            text = text.replace(char, replacement)
        # Catch any remaining non-latin1 chars
        return text.encode('latin-1', errors='replace').decode('latin-1')

    def _make_pdf(self, all_items, project_name: str):
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()

        # ── Cover Header ──────────────────────────────────────
        pdf.set_fill_color(15, 23, 42)
        pdf.rect(0, 0, 210, 50, 'F')

        pdf.set_font("Helvetica", "B", 22)
        pdf.set_text_color(255, 255, 255)
        pdf.set_xy(10, 10)
        pdf.cell(0, 10, "PROJECT EXIT BRIEF", ln=True, align="C")

        pdf.set_font("Helvetica", "", 13)
        pdf.set_text_color(200, 220, 255)
        pdf.set_xy(10, 23)
        pdf.cell(0, 8, self._sanitize(f"{project_name}  |  {self.team_name}"), ln=True, align="C")

        pdf.set_font("Helvetica", "I", 10)
        pdf.set_text_color(150, 180, 220)
        pdf.set_xy(10, 34)
        pdf.cell(0, 8, f"Generated by Xenone  |  {datetime.date.today().strftime('%B %d, %Y')}", ln=True, align="C")

        # ── Summary Stats ────────────────────────────────────
        stats = {}
        for item in all_items:
            cat = item["metadata"].get("category", "general")
            stats[cat] = stats.get(cat, 0) + 1

        pdf.set_fill_color(248, 250, 252)
        pdf.rect(10, 58, 190, 25, 'F')

        stat_labels = [
            ("Total Items", len(all_items)),
            ("Decisions", stats.get("decision", 0)),
            ("Lessons", stats.get("lesson", 0)),
            ("Warnings", stats.get("warning", 0)),
        ]
        for i, (label, count) in enumerate(stat_labels):
            x = 15 + i * 47
            pdf.set_font("Helvetica", "B", 18)
            pdf.set_text_color(15, 23, 42)
            pdf.set_xy(x, 61)
            pdf.cell(40, 8, str(count), align="C")
            pdf.set_font("Helvetica", "", 8)
            pdf.set_text_color(100, 120, 140)
            pdf.set_xy(x, 71)
            pdf.cell(40, 5, label.upper(), align="C")

        pdf.set_xy(10, 88)
        pdf.set_font("Helvetica", "I", 9)
        pdf.set_text_color(120, 140, 160)
        pdf.cell(0, 6, "Auto-generated from team knowledge captured via Xenone.", ln=True)
        pdf.ln(4)

        # ── Sections ─────────────────────────────────────────
        section_order = ["decision", "lesson", "warning", "context", "general"]
        seen_sections = set()

        for cat in section_order:
            if cat in seen_sections:
                continue
            if cat == "general":
                items = [i for i in all_items if i["metadata"].get("category") in ("general", "context")]
            else:
                items = [i for i in all_items if i["metadata"].get("category") == cat]

            if not items:
                continue

            seen_sections.add(cat)
            seen_sections.add("context")
            cfg = self.SECTION_CONFIG.get(cat, self.SECTION_CONFIG["general"])
            r, g, b = cfg["color"]
            br, bg_c, bb = cfg["bg"]

            # Section header bar
            pdf.set_fill_color(r, g, b)
            pdf.rect(10, pdf.get_y(), 190, 12, 'F')
            pdf.set_font("Helvetica", "B", 13)
            pdf.set_text_color(255, 255, 255)
            pdf.set_x(10)
            pdf.cell(190, 12, f"  {cfg['title']}  ({len(items)} items)", ln=True)
            pdf.ln(3)

            for item in items:
                meta = item["metadata"]
                y_before = pdf.get_y()

                pdf.set_fill_color(br, bg_c, bb)
                pdf.rect(10, y_before, 190, 30, 'F')

                # Meta line
                pdf.set_font("Helvetica", "B", 8)
                pdf.set_text_color(r, g, b)
                pdf.set_xy(12, y_before + 2)
                date_str = meta.get("timestamp", "")[:10]
                meta_line = self._sanitize(
                    f"{meta.get('author','?')}  |  {meta.get('channel','?')}  |  {date_str}  |  Tags: {meta.get('tags','')}"
                )
                pdf.cell(0, 5, meta_line, ln=True)

                # Content
                pdf.set_font("Helvetica", "", 10)
                pdf.set_text_color(20, 30, 50)
                pdf.set_x(12)
                wrapped = textwrap.fill(self._sanitize(item["content"]), width=100)
                for line in wrapped.split("\n"):
                    pdf.cell(0, 5, line, ln=True)

                pdf.ln(4)
            pdf.ln(4)

        # ── Footer ───────────────────────────────────────────
        pdf.set_y(-20)
        pdf.set_font("Helvetica", "I", 9)
        pdf.set_text_color(140, 150, 160)
        pdf.cell(0, 10,
            f"Xenone | AMD Slingshot | {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')} | Page {pdf.page_no()}",
            align="C"
        )
        return pdf

    def generate(self, project_name="Project Handoff", output_path="/content/xenone_exports/exit_brief.pdf"):
        print(f"\n📄 Generating Exit Brief for: {project_name}")
        all_items = self.kb.get_all()
        if not all_items:
            print("❌ No knowledge items found.")
            return None
        print(f"   Processing {len(all_items)} items...")
        pdf = self._make_pdf(all_items, project_name)
        pdf.output(output_path)
        size_kb = os.path.getsize(output_path) / 1024
        print(f"✅ Done! Saved to: {output_path} ({size_kb:.1f} KB)")
        print("💡 Files panel (left sidebar) → xenone_exports → exit_brief.pdf → Download")
        return output_path


# Re-initialize and re-run
exit_brief_gen = XenoneExitBrief(kb, CONFIG["team_name"])

pdf_path = exit_brief_gen.generate(
    project_name="Annual Business Plan Competition 2024",
    output_path="/content/xenone_exports/exit_brief.pdf"
)


📄 Generating Exit Brief for: Annual Business Plan Competition 2024
   Processing 13 items...
✅ Done! Saved to: /content/xenone_exports/exit_brief.pdf (5.2 KB)
💡 Files panel (left sidebar) → xenone_exports → exit_brief.pdf → Download


/tmp/ipython-input-514/2514509519.py:57: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "PROJECT EXIT BRIEF", ln=True, align="C")
/tmp/ipython-input-514/2514509519.py:62: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 8, self._sanitize(f"{project_name}  |  {self.team_name}"), ln=True, align="C")
/tmp/ipython-input-514/2514509519.py:67: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 8, f"Generated by Xenone  |  {datetime.date.today().strftime('%B %d, %Y')}", ln=True, align="C")
/tmp/ipython-input-514/2514509519.py:98: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 6, "Auto-generated from team knowledge captured vi

In [14]:
# ============================================================
# STEP 6B: Generate the Exit Brief!
# Click Run → PDF appears in /content/xenone_exports/
# ============================================================

# ── EDIT PROJECT NAME ──────────────────────────────────────
PROJECT_NAME = "Annual Business Plan Competition 2024"
# ───────────────────────────────────────────────────────────

pdf_path = exit_brief_gen.generate(
    project_name=PROJECT_NAME,
    output_path="/content/xenone_exports/exit_brief.pdf"
)

# Verify the file was created
if pdf_path and os.path.exists(pdf_path):
    print(f"\n🎉 SUCCESS! PDF is ready to download.")
    print(f"   Go to Files panel → xenone_exports → exit_brief.pdf → Download")


📄 Generating Exit Brief for: Annual Business Plan Competition 2024
   Processing 13 items...
✅ Done! Saved to: /content/xenone_exports/exit_brief.pdf (5.2 KB)
💡 Files panel (left sidebar) → xenone_exports → exit_brief.pdf → Download

🎉 SUCCESS! PDF is ready to download.
   Go to Files panel → xenone_exports → exit_brief.pdf → Download


/tmp/ipython-input-514/2514509519.py:57: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "PROJECT EXIT BRIEF", ln=True, align="C")
/tmp/ipython-input-514/2514509519.py:62: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 8, self._sanitize(f"{project_name}  |  {self.team_name}"), ln=True, align="C")
/tmp/ipython-input-514/2514509519.py:67: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 8, f"Generated by Xenone  |  {datetime.date.today().strftime('%B %d, %Y')}", ln=True, align="C")
/tmp/ipython-input-514/2514509519.py:98: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 6, "Auto-generated from team knowledge captured vi

## 🖥️ STEP 7: Interactive Gradio Dashboard
Full interactive UI — simulates the Xenone web dashboard running inside Colab

In [15]:
# ============================================================
# STEP 7: Gradio Interactive Dashboard
# All three Xenone features in one tabbed UI
# ============================================================

import gradio as gr

# ── Handler: Capture ──────────────────────────────────────
def handle_capture(content, author, channel, platform, category):
    if not content.strip() or not author.strip():
        return "❌ Content and Author are required."
    msg = capturer.capture_message(
        content=content,
        author=author,
        channel=channel,
        platform=platform,
        category=category if category != "auto-detect" else None
    )
    stats = kb.stats()
    return (
        f"✅ Saved to knowledge base!\n"
        f"ID: {msg.id}\n"
        f"Category: {msg.category}\n"
        f"Total items in KB: {stats['total']}\n\n"
        f"Breakdown: {json.dumps(stats['by_category'], indent=2)}"
    )

# ── Handler: Q&A ──────────────────────────────────────────
def handle_ask(question):
    if not question.strip():
        return "❌ Please enter a question.", ""
    result = qa.ask(question)

    confidence_bar = "█" * int(result.confidence_score * 20) + "░" * (20 - int(result.confidence_score * 20))

    answer_text = (
        f"💬 {result.answer}\n\n"
        f"📊 Confidence: {result.confidence_label} [{confidence_bar}] {result.confidence_score:.0%}\n"
        f"📎 {result.source_count} source(s) found"
    )

    sources_text = ""
    for i, src in enumerate(result.sources, 1):
        meta = src["metadata"]
        sources_text += (
            f"[Source {i}] {meta.get('author','?')} | {meta.get('channel','?')} | "
            f"{meta.get('timestamp','?')[:10]} | Relevance: {src['similarity']:.0%}\n"
            f"{src['content']}\n\n"
        )

    return answer_text, sources_text or "No sources found."

# ── Handler: Exit Brief ───────────────────────────────────
def handle_generate_brief(project_name):
    if not project_name.strip():
        project_name = "Team Handoff"
    path = exit_brief_gen.generate(
        project_name=project_name,
        output_path="/content/xenone_exports/exit_brief.pdf"
    )
    if path:
        stats = kb.stats()
        return (
            f"✅ Exit Brief generated!\n"
            f"Project: {project_name}\n"
            f"Items: {stats['total']}\n"
            f"File: {path}\n\n"
            f"Download: Files panel → xenone_exports → exit_brief.pdf",
            path
        )
    return "❌ Generation failed.", None

# ── Handler: Stats ─────────────────────────────────────────
def handle_stats():
    stats = kb.stats()
    all_items = kb.get_all()

    output = f"📊 KNOWLEDGE BASE STATS\n{'='*40}\n"
    output += f"Total items: {stats['total']}\n\n"
    output += "By category:\n"
    for cat, count in stats['by_category'].items():
        output += f"  {cat.capitalize():15}: {count}\n"
    output += f"\n{'='*40}\n"
    output += "Recent captures:\n"
    for item in all_items[-5:]:
        meta = item["metadata"]
        output += f"  [{meta.get('category','?')}] {meta.get('author','?')}: {item['content'][:60]}...\n"
    return output


# ── Build Gradio UI ───────────────────────────────────────
with gr.Blocks(
    title="Xenone — Team Knowledge Hub",
    theme=gr.themes.Soft(primary_hue="blue")
) as demo:
    gr.Markdown("""
    # 🧠 Xenone — Team Knowledge Hub
    ### AMD Slingshot Hackathon | *"Preserving 5 years of club knowledge in 5 clicks"*
    """)

    with gr.Tabs():

        # ── Tab 1: Capture ─────────────────────────────────
        with gr.Tab("📌 Capture Knowledge"):
            gr.Markdown("""
            ### Simulate pinning a message in Slack/Discord
            In production: reacting with 📌 triggers this automatically.
            """)
            with gr.Row():
                with gr.Column():
                    cap_content  = gr.Textbox(label="Message Content", lines=4, placeholder="Paste the message you want to capture...")
                    cap_author   = gr.Textbox(label="Author", placeholder="e.g. Samridhi")
                    cap_channel  = gr.Textbox(label="Channel", placeholder="e.g. #decisions", value="#general")
                    cap_platform = gr.Dropdown(["slack", "discord"], label="Platform", value="slack")
                    cap_category = gr.Dropdown(["auto-detect", "decision", "lesson", "warning", "context"], label="Category", value="auto-detect")
                    cap_btn      = gr.Button("📌 Capture Message", variant="primary")
                cap_output = gr.Textbox(label="Result", lines=8)
            cap_btn.click(handle_capture, inputs=[cap_content, cap_author, cap_channel, cap_platform, cap_category], outputs=cap_output)

        # ── Tab 2: Q&A ─────────────────────────────────────
        with gr.Tab("🤖 Ask Anything (/ask)"):
            gr.Markdown("""
            ### Ask questions about your team's knowledge
            In Slack/Discord: `/ask why did we choose our vendor?`
            """)
            with gr.Row():
                qa_question = gr.Textbox(label="Your Question", placeholder="Why did we choose our printing vendor?", scale=4)
                qa_btn = gr.Button("🔍 Ask", variant="primary", scale=1)
            with gr.Row():
                qa_answer  = gr.Textbox(label="Answer + Confidence Score", lines=6)
                qa_sources = gr.Textbox(label="Source Messages", lines=10)
            qa_btn.click(handle_ask, inputs=[qa_question], outputs=[qa_answer, qa_sources])
            gr.Examples(
                examples=[
                    ["Why did we choose our printing vendor?"],
                    ["What was the Q3 marketing budget?"],
                    ["What warnings should the new team know about?"],
                    ["How do I get budget approval from the student council?"],
                ],
                inputs=[qa_question]
            )

        # ── Tab 3: Exit Brief ──────────────────────────────
        with gr.Tab("📄 Exit Brief Generator"):
            gr.Markdown("""
            ### Generate a structured PDF handoff document
            Compiles all captured decisions, lessons, warnings, and context into one PDF.
            """)
            brief_name = gr.Textbox(label="Project Name", value="Annual Business Plan Competition 2024")
            brief_btn  = gr.Button("📄 Generate Exit Brief", variant="primary")
            with gr.Row():
                brief_output = gr.Textbox(label="Status", lines=6)
                brief_file   = gr.File(label="Download PDF")
            brief_btn.click(handle_generate_brief, inputs=[brief_name], outputs=[brief_output, brief_file])

        # ── Tab 4: Stats ───────────────────────────────────
        with gr.Tab("📊 Knowledge Base Stats"):
            gr.Markdown("### View all knowledge in your team's database")
            stats_btn    = gr.Button("🔄 Refresh Stats", variant="secondary")
            stats_output = gr.Textbox(label="Knowledge Base Overview", lines=20)
            stats_btn.click(handle_stats, outputs=stats_output)


# Launch the dashboard
print("🚀 Launching Xenone Dashboard...")
demo.launch(share=True, debug=False)

/tmp/ipython-input-514/734426927.py:92: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(


🚀 Launching Xenone Dashboard...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://42c067cd7e4670db4c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 🎬 STEP 8: Full End-to-End Demo
Complete walkthrough matching the AMD Slingshot demo script

In [16]:
# ============================================================
# STEP 8: Full End-to-End Demo
# Matches the AMD Slingshot Hackathon demo walkthrough
# ============================================================

print("\n" + "="*65)
print(" 🎬 XENONE FULL DEMO — AMD Slingshot Hackathon")
print("="*65)

# ── Scene Setup ────────────────────────────────────────────
print("""
SCENARIO:
An entrepreneurship club is wrapping up their annual competition.
The outgoing team lead is graduating. The incoming team has NO
context on vendor decisions, budget discussions, or lessons.
""")
input("▶ Press Enter to start Demo Step 1: Passive Capture...")

# ── Demo Step 1: Capture ───────────────────────────────────
print("\n" + "─"*60)
print("STEP 1 — PASSIVE CAPTURE (0:00 – 0:20)")
print("─"*60)
print("[Slack channel: #product-decisions]")
print("Sarah: 'We've decided to increase the Q3 marketing budget")
print("        by 20% and focus spend on LinkedIn & YouTube for B2B.'")
print("")
print("[Sarah reacts to the message with 📌]")
print("")

demo_msg = capturer.capture_message(
    content="We've decided to increase the Q3 marketing budget by 20% and focus spend on LinkedIn and YouTube for B2B audience targeting. This is based on our Q2 analysis showing LinkedIn generates 3x more qualified leads than Instagram for B2B.",
    author="Sarah L.",
    channel="#product-decisions",
    platform="slack",
    tags=["budget", "marketing", "Q3", "decision"],
    category="decision"
)
print("\n✅ Bot response in Slack: 'Saved to knowledge base | Tagged: budget · marketing · Q3 · decision'")

input("\n▶ Press Enter for Step 2: AI Q&A...")

# ── Demo Step 2: AI Q&A ────────────────────────────────────
print("\n" + "─"*60)
print("STEP 2 — AI Q&A WITH CONFIDENCE (0:20 – 0:55)")
print("─"*60)
print("New member types in Slack: /ask what was our marketing budget decision?")
print()

result = qa.ask("what was our marketing budget decision?")
qa.display_result(result)

input("\n▶ Press Enter for Step 3: Exit Brief...")

# ── Demo Step 3: Exit Brief ────────────────────────────────
print("\n" + "─"*60)
print("STEP 3 — EXIT BRIEF GENERATOR (0:55 – 1:25)")
print("─"*60)
print("[Team lead opens Xenone dashboard, clicks 'Generate Exit Brief']")
print()

final_pdf = exit_brief_gen.generate(
    project_name="Annual Business Plan Competition 2024",
    output_path="/content/xenone_exports/DEMO_exit_brief.pdf"
)

stats = kb.stats()
print(f"\n📊 Dashboard shows:")
print(f"   {stats['by_category'].get('decision',0)} DECISIONS CAPTURED")
print(f"   {stats['by_category'].get('lesson',0)} LESSONS LEARNED")
print(f"   {stats['by_category'].get('warning',0)} WARNINGS FLAGGED")

print("""
─────────────────────────────────────────────────────────────
  DEMO COMPLETE ✅

  ✅ Step 1: Knowledge captured passively via emoji reaction
  ✅ Step 2: Q&A answered from real team history with confidence
  ✅ Step 3: Exit brief PDF auto-generated in seconds

  "Once context is lost, it cannot be reconstructed."
  Xenone prevents that. Powered by AMD MI300X + ROCm + Llama 3
─────────────────────────────────────────────────────────────
""")


 🎬 XENONE FULL DEMO — AMD Slingshot Hackathon

SCENARIO:
An entrepreneurship club is wrapping up their annual competition.
The outgoing team lead is graduating. The incoming team has NO
context on vendor decisions, budget discussions, or lessons.

▶ Press Enter to start Demo Step 1: Passive Capture...

────────────────────────────────────────────────────────────
STEP 1 — PASSIVE CAPTURE (0:00 – 0:20)
────────────────────────────────────────────────────────────
[Slack channel: #product-decisions]
Sarah: 'We've decided to increase the Q3 marketing budget
        by 20% and focus spend on LinkedIn & YouTube for B2B.'

[Sarah reacts to the message with 📌]

✅ Saved to knowledge base
   📝 Content  : We've decided to increase the Q3 marketing budget by 20% and focus spend on Link...
   👤 Author   : Sarah L.
   📍 Channel  : #product-decisions (slack)
   🏷️  Category : decision
   🆔 ID       : 22452a4e
   📊 Total KB : 14 items

✅ Bot response in Slack: 'Saved to knowledge base | Tagged: budget

/tmp/ipython-input-514/2514509519.py:57: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "PROJECT EXIT BRIEF", ln=True, align="C")
/tmp/ipython-input-514/2514509519.py:62: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 8, self._sanitize(f"{project_name}  |  {self.team_name}"), ln=True, align="C")
/tmp/ipython-input-514/2514509519.py:67: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 8, f"Generated by Xenone  |  {datetime.date.today().strftime('%B %d, %Y')}", ln=True, align="C")
/tmp/ipython-input-514/2514509519.py:98: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 6, "Auto-generated from team knowledge captured vi

## 🔧 BONUS: Production Slack Bot Skeleton
Drop this into your production environment with a real Slack token

In [17]:
# ============================================================
# BONUS: Production Slack Bot Code
# This is the REAL bot logic — requires SLACK_BOT_TOKEN
# In Colab: shows the code only (cannot run without token)
# To deploy: copy to your server + set environment variables
# ============================================================

SLACK_BOT_SKELETON = '''
# ─────────────────────────────────────────────────────────
# xenone_slack_bot.py — Production Slack Bot
# ─────────────────────────────────────────────────────────
# Setup:
#   pip install slack-bolt
#   export SLACK_BOT_TOKEN=xoxb-...
#   export SLACK_APP_TOKEN=xapp-...
#   python xenone_slack_bot.py
# ─────────────────────────────────────────────────────────

import os
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler

# Import your Xenone modules
from xenone_kb import XenoneKnowledgeBase, CONFIG
from xenone_capture import XenoneCapture
from xenone_qa import XenoneQA
from xenone_brief import XenoneExitBrief

app = App(token=os.environ["SLACK_BOT_TOKEN"])
kb = XenoneKnowledgeBase(CONFIG)
capturer = XenoneCapture(kb)
qa_engine = XenoneQA(kb, ...)

# ── Capture: React with 📌 to any message ──────────────────
@app.event("reaction_added")
def handle_pin_reaction(event, client, say):
    if event["reaction"] != "pushpin":
        return

    # Fetch the original message text
    result = client.conversations_history(
        channel=event["item"]["channel"],
        latest=event["item"]["ts"],
        limit=1,
        inclusive=True
    )
    message = result["messages"][0]
    user_info = client.users_info(user=message["user"])
    author = user_info["user"]["real_name"]

    capturer.capture_message(
        content=message["text"],
        author=author,
        channel=event["item"]["channel"],
        platform="slack"
    )

    # Confirm capture in the channel
    client.reactions_add(
        channel=event["item"]["channel"],
        timestamp=event["item"]["ts"],
        name="white_check_mark"
    )

# ── Q&A: /ask command ──────────────────────────────────────
@app.command("/ask")
def handle_ask_command(ack, respond, command):
    ack()
    question = command["text"]
    result = qa_engine.ask(question)

    respond({
        "blocks": [
            {"type": "section", "text": {"type": "mrkdwn",
                "text": f"*Answer:* {result.answer}"}},
            {"type": "section", "text": {"type": "mrkdwn",
                "text": f"*Confidence:* {result.confidence_label} ({result.confidence_score:.0%}) | *Sources:* {result.source_count}"}},
        ]
    })

# ── Handoff: /handoff command ──────────────────────────────
@app.command("/handoff")
def handle_handoff(ack, respond, command):
    ack()
    project_name = command["text"] or "Project Handoff"
    brief_gen = XenoneExitBrief(kb, "My Team")
    path = brief_gen.generate(project_name)
    respond(f"✅ Exit Brief generated for *{project_name}*. Check your DMs!")

if __name__ == "__main__":
    handler = SocketModeHandler(app, os.environ["SLACK_APP_TOKEN"])
    handler.start()
'''

print("📋 Production Slack Bot Code:")
print("="*60)
print(SLACK_BOT_SKELETON)
print("="*60)
print("\n💾 Saving skeleton to file...")

with open("/content/xenone_exports/xenone_slack_bot_skeleton.py", "w") as f:
    f.write(SLACK_BOT_SKELETON)

print("✅ Saved to /content/xenone_exports/xenone_slack_bot_skeleton.py")
print("   Download it from the Files panel!")

📋 Production Slack Bot Code:

# ─────────────────────────────────────────────────────────
# xenone_slack_bot.py — Production Slack Bot
# ─────────────────────────────────────────────────────────
# Setup:
#   pip install slack-bolt
#   export SLACK_BOT_TOKEN=xoxb-...
#   export SLACK_APP_TOKEN=xapp-...
#   python xenone_slack_bot.py
# ─────────────────────────────────────────────────────────

import os
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler

# Import your Xenone modules
from xenone_kb import XenoneKnowledgeBase, CONFIG
from xenone_capture import XenoneCapture
from xenone_qa import XenoneQA
from xenone_brief import XenoneExitBrief

app = App(token=os.environ["SLACK_BOT_TOKEN"])
kb = XenoneKnowledgeBase(CONFIG)
capturer = XenoneCapture(kb)
qa_engine = XenoneQA(kb, ...)

# ── Capture: React with 📌 to any message ──────────────────
@app.event("reaction_added")
def handle_pin_reaction(event, client, say):
    if event["reaction"] != "pushpin":

## 🚀 BONUS: AMD MI300X Production Setup
How to run Xenone on AMD Developer Cloud with Ollama + Llama 3

In [18]:
# ============================================================
# BONUS: AMD MI300X + Ollama Setup Commands
# Run these on your AMD Developer Cloud instance
# NOT runnable in Colab (no AMD GPU here)
# ============================================================

AMD_SETUP = """
╔══════════════════════════════════════════════════════════╗
║   AMD MI300X PRODUCTION SETUP — Xenone                  ║
╠══════════════════════════════════════════════════════════╣

1. PROVISION AMD DEVELOPER CLOUD INSTANCE
   - Go to: https://developer.amd.com/amd-developer-cloud/
   - Create account → get $100 free credits
   - Launch MI300X instance (GPU-enabled)

2. INSTALL ROCm (AMD's GPU software stack)
   wget https://repo.radeon.com/amdgpu-install/latest/ubuntu/focal/amdgpu-install_5.7.50700-1_all.deb
   sudo dpkg -i amdgpu-install_5.7.50700-1_all.deb
   sudo amdgpu-install --usecase=rocm
   rocm-smi  # verify GPU is detected

3. INSTALL OLLAMA (runs Llama 3 on AMD GPU via ROCm)
   curl -fsSL https://ollama.ai/install.sh | sh
   ROCM_PATH=/opt/rocm ollama serve &
   ollama pull llama3          # 8B model (~5GB)
   ollama pull llama3:70b      # 70B model (best quality)

4. REPLACE T5 WITH OLLAMA IN XENONE
   # In xenone_qa.py, replace _generate_answer():
   import ollama
   def _generate_answer(self, prompt):
       response = ollama.generate(
           model='llama3',
           prompt=prompt,
           options={'temperature': 0.3, 'max_tokens': 200}
       )
       return response['response']

5. INSTALL PYTHON DEPS + RUN
   pip install chromadb sentence-transformers fastapi uvicorn
   pip install slack-bolt fpdf2
   python xenone_bot.py &
   uvicorn xenone_api:app --host 0.0.0.0 --port 8000

6. PERFORMANCE ON MI300X
   - 1.3 PetaFLOPs AI performance
   - 192GB HBM3 memory
   - Llama 3 70B: ~2-3s response time
   - Cost: ~$0.50/hour (vs $0.02/query for OpenAI)
   - Break-even: >25 queries/hour → unlimited value

╚══════════════════════════════════════════════════════════╝
"""

print(AMD_SETUP)


╔══════════════════════════════════════════════════════════╗
║   AMD MI300X PRODUCTION SETUP — Xenone                  ║
╠══════════════════════════════════════════════════════════╣

1. PROVISION AMD DEVELOPER CLOUD INSTANCE
   - Go to: https://developer.amd.com/amd-developer-cloud/
   - Create account → get $100 free credits
   - Launch MI300X instance (GPU-enabled)

2. INSTALL ROCm (AMD's GPU software stack)
   wget https://repo.radeon.com/amdgpu-install/latest/ubuntu/focal/amdgpu-install_5.7.50700-1_all.deb
   sudo dpkg -i amdgpu-install_5.7.50700-1_all.deb
   sudo amdgpu-install --usecase=rocm
   rocm-smi  # verify GPU is detected

3. INSTALL OLLAMA (runs Llama 3 on AMD GPU via ROCm)
   curl -fsSL https://ollama.ai/install.sh | sh
   ROCM_PATH=/opt/rocm ollama serve &
   ollama pull llama3          # 8B model (~5GB)
   ollama pull llama3:70b      # 70B model (best quality)

4. REPLACE T5 WITH OLLAMA IN XENONE
   # In xenone_qa.py, replace _generate_answer():
   import ollama
   de

---
## 📋 Quick Reference

| Feature | Colab Command | Production (Slack) |
|---------|-------------|-------------------|
| Capture a message | `capturer.capture_message(content, author, channel)` | React with 📌 emoji |
| Ask a question | `result = qa.ask("your question")` | `/ask your question` |
| Generate exit brief | `exit_brief_gen.generate("Project Name")` | `/handoff Project Name` |
| View all knowledge | `kb.get_all()` | Dashboard → Stats tab |
| Search knowledge | `kb.search("query")` | `/ask query` |

## 🔑 Environment Variables Needed (Production)
```
SLACK_BOT_TOKEN=xoxb-...       # From Slack App settings
SLACK_APP_TOKEN=xapp-...       # Enable Socket Mode in Slack
DISCORD_TOKEN=...              # From Discord Developer Portal
XENONE_DB_PATH=/data/xenone    # Persistent storage path
OLLAMA_HOST=http://localhost:11434  # Ollama API endpoint
```

## 📁 Project Structure
```
xenone/
├── xenone_kb.py          # ChromaDB knowledge base
├── xenone_capture.py     # Passive capture module
├── xenone_qa.py          # RAG Q&A engine
├── xenone_brief.py       # Exit brief PDF generator
├── xenone_slack_bot.py   # Slack bot
├── xenone_discord_bot.py # Discord bot
├── xenone_api.py         # FastAPI REST endpoints
└── xenone_dashboard.py   # Streamlit dashboard
```